In [65]:
import pandas as pd
import numpy as np
import country_converter as coco

In [66]:
elevate_scens = pd.read_csv(
    r"X:\user\dekkerm\Data\ELEVATE\ELEVATE_Data_D2.3_vetted_20250211.csv"
)
engage_scens = pd.read_csv(
    r"X:\user\dekkerm\Data\ENGAGE\PolicyScenarios\ENGAGE_internal_2610_onlyemis.csv"
)

In [67]:
# Filter for variables and scenarios
variables = ["Emissions|Kyoto Gases", "Emissions|CO2"]
# scenarios = {"CurPol": "ELV-SSP2-CP-D0", "CurPol_national": "ELV-SSP2-CP-D0", "NDC": "ELV-SSP2-NDC-D0", "NetZero": "ELV-SSP2-LTS"}
scenarios = {
    "ELV-SSP2-CP-D0": "CurPol",
    "ELV-SSP2-CP-D0-N": "CurPol",
    "Current Policies": "CurPol",
    "ELV-SSP2-NDC-D0": "NDC",
    "ELV-SSP2-LTS": "NetZero"
}
elevate_scens_filtered = elevate_scens[elevate_scens["Variable"].isin(variables)]
elevate_scens_filtered = elevate_scens_filtered[elevate_scens_filtered["Scenario"].isin(scenarios.keys())]

# Rename columns: Remove leading 'X' from year columns
elevate_scens_filtered.columns = [
    col[1:] if col.startswith("X") and col[1:].isdigit() else col
    for col in elevate_scens_filtered.columns
]

#Rename scenarios
elevate_scens_filtered["Scenario"] = elevate_scens_filtered["Scenario"].replace(scenarios)

### Expand regions

In [68]:
# Split the column by '|' and expand into new columns
split_columns = elevate_scens_filtered['Region'].str.split('|', expand=True)

# Rename the new columns (optional)
split_columns.columns = ['Model_2', 'Region_2']

# Add the new columns to the original DataFrame
elevate_scens_filtered = pd.concat([elevate_scens_filtered, split_columns], axis=1)

In [69]:
elevate_scens_filtered.columns

Index(['Model', 'Scenario', 'Region', 'Variable', 'Unit', '1990', '1995',
       '2000', '2005', '2010', '2015', '2016', '2017', '2020', '2021', '2025',
       '2030', '2035', '2040', '2045', '2050', '2055', '2060', '2065', '2070',
       '2075', '2080', '2085', '2090', '2095', '2100', 'Model_2', 'Region_2'],
      dtype='object')

In [70]:
#Change order of columns
elevate_scens_filtered = elevate_scens_filtered[['Model', 'Model_2', 'Scenario', 'Region', 'Region_2', 'Variable', 'Unit', '1990', '1995',
       '2000', '2005', '2010', '2015', '2016', '2017', '2020', '2021', '2025',
       '2030', '2035', '2040', '2045', '2050', '2055', '2060', '2065', '2070',
       '2075', '2080', '2085', '2090', '2095', '2100']]

In [71]:
# Replace values in Model_2 with NaN if they are the same as in Model to filter out regions
elevate_scens_filtered['Model_2'] = np.where(
    elevate_scens_filtered['Model_2'] == elevate_scens_filtered['Model'],
    np.nan,
    elevate_scens_filtered['Model_2']
)

In [72]:
# Merge the two columns into a new column 'Region_cleaned', drop old columns and rearrange order
elevate_scens_filtered["Region_cleaned"] = elevate_scens_filtered[
    "Model_2"
].combine_first(elevate_scens_filtered["Region_2"])



In [73]:
elevate_scens_filtered.columns

Index(['Model', 'Model_2', 'Scenario', 'Region', 'Region_2', 'Variable',
       'Unit', '1990', '1995', '2000', '2005', '2010', '2015', '2016', '2017',
       '2020', '2021', '2025', '2030', '2035', '2040', '2045', '2050', '2055',
       '2060', '2065', '2070', '2075', '2080', '2085', '2090', '2095', '2100',
       'Region_cleaned'],
      dtype='object')

In [74]:
#Rename regions
elevate_scens_filtered["Region_cleaned"] = elevate_scens_filtered[
    "Region_cleaned"
].replace("United States of America", "USA")
elevate_scens_filtered["Region_cleaned"] = elevate_scens_filtered[
    "Region_cleaned"
].replace(["Southeast Asia", "South-East Asia", "South East Asia"], "Southeast Asia")
elevate_scens_filtered["Region_cleaned"] = elevate_scens_filtered[
    "Region_cleaned"
].replace("World", "EARTH")
elevate_scens_filtered["Region_cleaned"] = elevate_scens_filtered[
    "Region_cleaned"
].replace("European Union", "EU")

elevate_scens_filtered.drop(columns=['Model_2', 'Region_2'], inplace=True)
# elevate_scens_filtered = elevate_scens_filtered[['Model', 'Scenario', 'Region', 'Region_cleaned', 'Variable', 'Unit', '1990', '1995',
#        '2000', '2005', '2010', '2015', '2016', '2017', '2020', '2021', '2025',
#        '2030', '2035', '2040', '2045', '2050', '2055', '2060', '2065', '2070',
#        '2075', '2080', '2085', '2090', '2095', '2100']]


# Dynamically select the first six columns and all year columns
columns_to_keep = [
    "Model",
    "Scenario",
    "Region",
    "Region_cleaned",
    "Variable",
    "Unit",
] + [col for col in elevate_scens_filtered.columns if col.isdigit()]

# Reorder the DataFrame
elevate_scens_filtered = elevate_scens_filtered[columns_to_keep]

In [75]:
# Convert countries to ISO3 codes and leave regions as is
cc = coco.CountryConverter()
elevate_scens_filtered["Region_cleaned"] = cc.pandas_convert(
    series=elevate_scens_filtered["Region_cleaned"], to="ISO3", not_found=None
)

EARTH not found in regex
Middle East not found in regex
Southeast Asia not found in regex
Africa (R10) not found in regex
Europe (R10) not found in regex
Latin America (R10) not found in regex
Middle East (R10) not found in regex
North America (R10) not found in regex
Pacific OECD (R10) not found in regex
Reforming Economies (R10) not found in regex
Rest of Asia (R10) not found in regex
EU not found in ISO2
Rest of Asia not found in regex
Rest of East and South East Asia not found in regex
Latin America and the Caribbean not found in regex
Middle East and North Africa not found in regex
Other Pacific Asia not found in regex
Rest of Centrally Planned Asia not found in regex
South Asia not found in regex
Other (R10) not found in regex
EU28 not found in regex
European Union (R9) not found in regex
USA (R9) not found in regex
More than one regular expression match for Australia, New Zealand, and Oceania islands
More than one regular expression match for Australia, New Zealand, and Oceania 

In [78]:
elevate_scens_filtered[["Region_cleaned"]].value_counts()

TypeError: unhashable type: 'list'

In [40]:

# Convert the 'Region' column to ISO3 country codes
elevate_scens_filtered["ISO3"] = coco.convert(
    names=elevate_scens_filtered["Region_cleaned"], to="ISO3", not_found=None
)

EARTH not found in regex
EARTH not found in regex
Middle East not found in regex
Middle East not found in regex
Southeast Asia not found in regex
Southeast Asia not found in regex
EARTH not found in regex
EARTH not found in regex
Middle East not found in regex
Middle East not found in regex
Southeast Asia not found in regex
Southeast Asia not found in regex
EARTH not found in regex
EARTH not found in regex
Africa (R10) not found in regex
Africa (R10) not found in regex
Europe (R10) not found in regex
Europe (R10) not found in regex
Latin America (R10) not found in regex
Latin America (R10) not found in regex
Middle East (R10) not found in regex
Middle East (R10) not found in regex
North America (R10) not found in regex
North America (R10) not found in regex
Pacific OECD (R10) not found in regex
Pacific OECD (R10) not found in regex
Reforming Economies (R10) not found in regex
Reforming Economies (R10) not found in regex
Rest of Asia (R10) not found in regex
Rest of Asia (R10) not found

In [80]:
# Filter rows where 'Region_cleaned' contains lists
rows_with_lists = elevate_scens_filtered[elevate_scens_filtered['Region_cleaned'].apply(lambda x: isinstance(x, list))]

# Display the rows with lists
rows_with_lists

,Model,Scenario,Region,Region_cleaned,Variable,Unit,1990,1995,2000,2005,...,2055,2060,2065,2070,2075,2080,2085,2090,2095,2100
60490,WITCH 5.0,CurPol,"WITCH 5.0|Australia, New Zealand, and Oceania ...","[AUS, NZL]",Emissions|Kyoto Gases,Mt CO2-equiv/yr,NaN,NaN,NaN,874.0600,...,787.43410,767.03780,757.83750,758.35590,764.08510,798.193300,843.209300,893.968800,961.584100,1040.938000
60514,WITCH 5.0,CurPol,"WITCH 5.0|Australia, New Zealand, and Oceania ...","[AUS, NZL]",Emissions|CO2,Mt CO2/yr,NaN,NaN,NaN,537.1571,...,483.17150,465.74000,456.34490,456.86590,463.55840,498.847000,545.390000,597.854400,668.085000,750.056500
68246,WITCH 5.0,NetZero,"WITCH 5.0|Australia, New Zealand, and Oceania ...","[AUS, NZL]",Emissions|Kyoto Gases,Mt CO2-equiv/yr,NaN,NaN,NaN,874.0600,...,65.83913,65.38783,65.54601,66.22279,66.22279,66.222790,66.222790,66.222790,66.222790,59.063980
68270,WITCH 5.0,NetZero,"WITCH 5.0|Australia, New Zealand, and Oceania ...","[AUS, NZL]",Emissions|CO2,Mt CO2/yr,NaN,NaN,NaN,537.1571,...,-187.59500,-194.37730,-202.47680,-220.18990,-233.16870,-245.326400,-251.844000,-261.314700,-267.373100,-276.257100
70188,WITCH 5.0,NDC,"WITCH 5.0|Australia, New Zealand, and Oceania ...","[AUS, NZL]",Emissions|Kyoto Gases,Mt CO2-equiv/yr,NaN,NaN,NaN,874.0600,...,780.28350,763.73360,754.30550,755.24900,701.05600,314.845800,308.326400,307.320700,305.804900,308.243300
70212,WITCH 5.0,NDC,"WITCH 5.0|Australia, New Zealand, and Oceania ...","[AUS, NZL]",Emissions|CO2,Mt CO2/yr,NaN,NaN,NaN,537.1571,...,482.22320,465.74000,456.34490,456.86590,402.91610,5.662748,5.662748,5.662748,5.662748,5.662748
84257,REMIND 3.3,CurPol,REMIND 3.3|China and Taiwan,"[CHN, TWN]",Emissions|Kyoto Gases,Mt CO2-equiv/yr,NaN,NaN,NaN,9909.0450,...,7059.09500,5822.97800,NaN,5090.69400,NaN,4466.043000,NaN,3756.885000,NaN,3032.904000
84289,REMIND 3.3,CurPol,REMIND 3.3|China and Taiwan,"[CHN, TWN]",Emissions|CO2,Mt CO2/yr,NaN,NaN,NaN,7669.5630,...,5204.53500,4248.94200,NaN,3712.90300,NaN,3230.073000,NaN,2638.754000,NaN,2015.114000
90923,REMIND 3.3,NetZero,REMIND 3.3|China and Taiwan,"[CHN, TWN]",Emissions|Kyoto Gases,Mt CO2-equiv/yr,NaN,NaN,NaN,9892.6750,...,1787.32900,907.66870,NaN,486.16900,NaN,284.194600,NaN,78.513490,NaN,24.246260
90955,REMIND 3.3,NetZero,REMIND 3.3|China and Taiwan,"[CHN, TWN]",Emissions|CO2,Mt CO2/yr,NaN,NaN,NaN,7653.1930,...,926.13170,134.52120,NaN,-219.05130,NaN,-312.655900,NaN,-426.144400,NaN,-415.923900


In [43]:
elevate_scens_filtered[['Region_cleaned']].value_counts()

Region_cleaned                             
India                                          76
China                                          70
Brazil                                         60
EARTH                                          46
Africa (R10)                                   42
Rest of Asia (R10)                             42
Reforming Economies (R10)                      42
Pacific OECD (R10)                             42
North America (R10)                            42
Latin America (R10)                            42
India+ (R10)                                   42
Middle East (R10)                              42
Europe (R10)                                   42
China+ (R10)                                   42
USA                                            36
Japan                                          32
EU                                             30
Indonesia                                      28
Middle East and North Africa                   18
Latin 

In [53]:
elevate_scens_filtered[['ISO3']]

,ISO3
527,EARTH
551,EARTH
591,BRA
615,BRA
655,CHN
...,...
112661,Other Southern Asia
112722,Southeast Asia
112761,Southeast Asia
117795,SAU


In [286]:
elevate_scens_filtered.columns

Index(['Model', 'Scenario', 'Region', 'Region_cleaned', 'Variable', 'Unit',
       '1990', '1995', '2000', '2005', '2010', '2015', '2016', '2017', '2020',
       '2021', '2025', '2030', '2035', '2040', '2045', '2050', '2055', '2060',
       '2065', '2070', '2075', '2080', '2085', '2090', '2095', '2100'],
      dtype='object')

In [ ]:
# Sort the dataframe by 'Region_cleaned' and reset the index
elevate_scens_filtered.sort_values(by=['Region_cleaned'], inplace=True)
elevate_scens_filtered.reset_index(drop=True, inplace=True)

# Add a new column 'Is_Duplicate' to indicate subsequent duplicates
elevate_scens_filtered['Is_Duplicate'] = elevate_scens_filtered.duplicated(subset=['Model', 'Scenario', 'Variable', '2025', '2100'], keep='first')

# Remove all rows that are Is_Duplicated = True
elevate_scens_deduplicated = elevate_scens_filtered[~elevate_scens_filtered['Is_Duplicate']]

elevate_scens_deduplicated.drop(columns=['Is_Duplicate', 'Region'], inplace=True)
elevate_scens_deduplicated.rename(columns={'Region_cleaned': 'Region'}, inplace=True)

C:\Users\wurschingc\AppData\Local\Temp\2\ipykernel_9480\1315033224.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  elevate_scens_deduplicated.drop(columns=['Is_Duplicate', 'Region'], inplace=True)
C:\Users\wurschingc\AppData\Local\Temp\2\ipykernel_9480\1315033224.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  elevate_scens_deduplicated.rename(columns={'Region_cleaned': 'Region'}, inplace=True)
